# Importation des données :

In [1]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [6]:
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import pandas as pd
import requests
import numpy as np 

In [7]:
OECD = pd.read_excel('données/OECD - short term economic indicators.xlsx')

In [8]:
IMF = pd.read_excel('données/IMF - monetary policy.xlsx')

In [10]:
agregated_data=pd.ExcelFile('données/FRBNY-SCE-Data.xlsx')
agregated_anticipation= df1=pd.read_excel(agregated_data, 'Inflation expectations')

In [16]:
agrant = agregated_anticipation[['Unnamed: 1','Unnamed: 2']].drop([0,1,2]).reset_index().rename(columns={'index':'date','Unnamed: 1':'short exp','Unnamed: 2':'mid exp'})

# Test de stationnarité :

## I - Tests sur les anticipations d'inflation : 

In [29]:
pip install arch

Note: you may need to restart the kernel to use updated packages.


In [30]:
from arch.unitroot import *

### A- Test ADF :

On importe de Statmodel la fonction pour réaliser un test adf. 

In [11]:
from statsmodels.tsa.stattools import adfuller

In [38]:
adfuller(agrant['short exp'])

(-1.6378044817107078,
 0.46340824386150187,
 4,
 109,
 {'1%': -3.49181775886872,
  '5%': -2.8884437992971588,
  '10%': -2.5811201893779985},
 -18.072284316318786)

La P-value est de 0.46. On ne rejette pas l'hypothèse nulle ( avec H0: la série n'est pas stationnaire). Ici, la série n'est donc pas stationnaire. 

In [46]:
adf_ct = ADF(agrant['short exp'].astype(float), trend = 'ct')
adf_ct.summary()

Test Statistic,-2.244
P-value,0.465
Lags,4


In [20]:
adfuller(agrant['mid exp'])

(-2.403988880387284,
 0.14061374036229007,
 2,
 111,
 {'1%': -3.490683082754047,
  '5%': -2.8879516565798817,
  '10%': -2.5808574442009578},
 -71.84627705214095)

Ici aussi, on ne rejette pas l'hypothèse nulle. La série n'est pas stationnaire. 

In [47]:
adf_ct = ADF(agrant['short exp'].astype(float), trend = 'ct')
adf_ct.summary()

Test Statistic,-2.244
P-value,0.465
Lags,4


### B - Test KPSS:

Ici, H0 = la série est stationnaire, H1= la série n'est pas stationnaire. 

In [21]:
import statsmodels.api as sm

In [22]:
sm.tsa.stattools.kpss(agrant['short exp'], regression='ct')

/opt/mamba/lib/python3.10/site-packages/statsmodels/tsa/stattools.py:2018: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  warnings.warn(


(0.3086618000884876,
 0.01,
 6,
 {'10%': 0.119, '5%': 0.146, '2.5%': 0.176, '1%': 0.216})

In [32]:
kpss_st = KPSS(agrant['short exp'], trend = 'ct')
kpss_st.summary()

Test Statistic,0.309
P-value,0.001
Lags,6


La p-value est inférieure à 0.01. On rejette l'hypothèse nulle au seuil de 1%. La série n'est donc pas stationnaire.

In [23]:
sm.tsa.stattools.kpss(agrant['mid exp'], regression='ct')

/opt/mamba/lib/python3.10/site-packages/statsmodels/tsa/stattools.py:2018: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  warnings.warn(


(0.2480908342699511,
 0.01,
 6,
 {'10%': 0.119, '5%': 0.146, '2.5%': 0.176, '1%': 0.216})

In [31]:
kpss_st = KPSS(agrant['mid exp'], trend = 'ct')
kpss_st.summary()

Test Statistic,0.248
P-value,0.005
Lags,6


La p-value est inférieure à 0.01. On rejette l'hypothèse nulle au seuil de 1%. La série n'est donc pas stationnaire.

### C - Test de Phillips Perron :

In [44]:
php_ct = PhillipsPerron(agrant['short exp'].astype(float), trend = 'ct')
php_ct.summary()

Test Statistic,-1.569
P-value,0.804
Lags,13


Ici, on ne rejette pas l'hypothèse nulle. La série n'est donc pas stationnaire. 

In [45]:
php_ct = PhillipsPerron(agrant['mid exp'].astype(float), trend = 'ct')
php_ct.summary()

Test Statistic,-2.628
P-value,0.267
Lags,13


Ici, on ne rejette pas l'hypothèse nulle. La série n'est donc pas stationnaire. 

# II - Tests sur l'inflation : 

In [48]:
Inflation = pd.read_excel('données/Inflation US.xlsx')

In [60]:
Inflation_finale=Inflation['tx_evol_ann_pct'].dropna( axis=0)

## A- Test ADF :

In [61]:
adf_inf = ADF(Inflation_finale.astype(float), trend = 'ct')
adf_inf.summary()

Test Statistic,-2.972
P-value,0.140
Lags,15


On ne rejette pas l'hypothèse nulle. La série n'est pas stationnaire. 

## B - Test KPSS :

In [62]:
kpss_st = KPSS(Inflation_finale, trend = 'ct')
kpss_st.summary()

Test Statistic,0.221
P-value,0.009
Lags,12


On rejette l'hypothèse nulle au seuil de 1 %. La série n'est donc pas stationnaire. 

## C - Test de phillips Perron : 

In [63]:
php_ct = PhillipsPerron(Inflation_finale.astype(float), trend = 'ct')
php_ct.summary()

Test Statistic,-3.434
P-value,0.047
Lags,18


On rejette l'hypothèse nulle au seuil de 5%. La série est faiblement stationnaire.